In [1]:
import requests
import json
import pandas as pd
import time
import numpy as np
from pandas.io.json import json_normalize

In [2]:
def date_transformation(time_stamp):
    return str(time_stamp).split()[0]

In [3]:
def time_transformation(time_stamp):
    return str(time_stamp).split()[1]

In [4]:
def url_count(urls):
    if type(urls) == type(list()):
        return len(urls)
    else:
        return 0

In [5]:
def user_mentions(users):
    if type(users) == type(list()):
        return len(users)
    else:
        return 0  

In [6]:
def media(list_of_media):
    if type(list_of_media) == type(list()) and len(list_of_media) > 0:
        return 1
    else:
        return 0

In [7]:
def is_reply(replies):
    if type(replies) == type(str()):
        return 1
    else:
        return 0    

In [8]:
# 'fromDate':'200603210000','toDate':'201904220000'
def twitter_Data(keywords,next_token,toDate):
    if next_token:
        params = {'query':delim.join(keywords),'maxResults':'500',\
                  'toDate':toDate,'next':next_token}
    else:
        params = {'query':delim.join(keywords),'maxResults':'500','toDate':toDate}
    r = requests.get(twitter_url,params=params,headers=header)
    data = r.json()
    try:
        user_cols = list(data['results'][0]['user'].keys())
        df_twitter = json_normalize(data['results'])
        df_twitter.created_at = pd.to_datetime(df_twitter.created_at,infer_datetime_format=True)
        df_twitter['date'] = df_twitter.created_at.agg(date_transformation)
        df_twitter['time'] = df_twitter.created_at.agg(time_transformation)
        df_twitter['urls'] = df_twitter['extended_tweet.entities.urls'].agg(url_count)
        df_twitter['user_mentions'] = df_twitter['entities.user_mentions'].agg(user_mentions)
        df_twitter['contains_media'] = df_twitter['extended_tweet.extended_entities.media'].agg(media)
        df_twitter['reply'] = df_twitter['in_reply_to_screen_name'].agg(is_reply)
        df_twitter['retweeted'] = df_twitter['retweeted']*1
        df_twitter['verified'] = df_twitter['user.verified']*1
        df_twitter['favorited'] = df_twitter['favorited']*1
        df_twitter['geo_enabled'] = df_twitter['user.geo_enabled']*1
        df_twitter['contributors_enabled'] = df_twitter['user.contributors_enabled']*1
        df_twitter['background_image'] = df_twitter['user.profile_use_background_image']*1
        df_cols = ['date','time','user.screen_name','text','user.followers_count','user.friends_count',\
                   'reply_count','retweet_count','user.favourites_count','verified','user.listed_count',\
                   'background_image','user.statuses_count','geo_enabled',\
                   'contributors_enabled','favorited','favorite_count','urls','user_mentions','reply',\
                   'reply_count']
        cols = [col for col in list(df_twitter.columns) if col not in df_cols]
        df_twitter = df_twitter.drop(cols,axis=1)
        if 'next' in list(data.keys()):
            next_token = data['next']
        return df_twitter,next_token
    except:
        return None,None

In [9]:
def stock_Data(ticker,key):
    r_2 = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+ticker+\
                   '&apikey='+key+'&outputsize=full')
    if r_2.status_code != 200:
        return None
    else:
        stock_data = r_2.json()
        if 'Time Series (Daily)' in list(stock_data.keys()):
            stock_data = stock_data['Time Series (Daily)']
            stocks = [{'date':day,'data':stock_data[day]} for day in stock_data]
            df_stocks = json_normalize(stocks)
            df_stocks['ticker'] = ticker
            df_stocks.columns = ['open','high','low','close','volume','date','ticker']
            date = df_stocks.date.max().split('-')
            toDate = date[0]+date[1]+date[2]+'0000'
            return df_stocks[df_stocks.date>'2006-03-21'],toDate
        else:
            return None,None

In [10]:
# keywords=[['AAPL','Apple'],['FB','Facebook'],['AMZN','Amazon'],['GOOGL','Google'],['MSFT','Microsoft'],\
#          ['WMT','Walmart'],['BBY','Best Buy'],['KR','Kroger'],['MCD',"McDonald's"],['HD','Home Depot'],\
#          ['NKE','Nike'],['ADDYY','Adidas'],['UAA','Under Armour'],\
#          ['JPM','JPMorgan'],['BRK.A','Berkshire Hathaway'],['GS','Goldman Sachs'],\
#          ['PGR','Progressive'],['ALL','Allstate'],['TRV','Travelers'],['AIG','American International Group'],\
#          ['MET','MetLife'],['UNH','United Healthcare'],['WLP','Wellpoint'],['AET','Aetna'],['CI','Cigna'],\
#          ['HUM','Humana'],['CNC','Centene'],['FCAU','Fiat','Chrysler'],['NSANY','Nissan','Motor'],\
#          ['F','Ford','Motor'],['TSLA','Tesla'],['GM','General','Motors'],['HMC','Honda','Motor'],['BMWYY','BMW'],\
#          ['DDAIF','Daimler','Mercedes'],['VWAGY','Volkswagen'],['TM','Toyota','Motor'],['SBUX','Starbucks'],\
#          ['YUM','Yum!','Brands'],['CMG','Chipotle'],['Burger','King'],['Olive','Garden'],["Domino's",'Pizza'],\
#          ['Panera','Bread'],['Dunkin','Donuts'],["Chili's"],['General','Mills'],['Kraft','Macaroni','Cheese'],\
#          ['Kellogs'],['Nestle'],['POST','Post','Holdings'],['McCormick','Company'],['Sony','Playstation'],['Nintendo'],\
#          ['Puma']]
# tickers = ['AAPL','FB','AMZN','GOOGL','MSFT',\
#           'WMT','BBY','KR','MCD','HD',\
#           'NKE','ADDYY','UAA',\
#           'JPM','BRK','GS',\
#           'PGR','ALL','TRV','AIG','MET',\
#           'UNH','WLP','AET','CI','HUM','CNC',\
#           'FCAU','NSANY','F','TSLA','GM','HMC','BMWYY','DDAIF','VWAGY','TM',\
#           'SBUX','YUM','CMG','QSR','DRI','DPZ','PNRA','DNKN','EAT',\
#           'GIS','KHC','K','NSRGY','POST','MKC',\
#           'SNE','NTDOY','PUMSY']
keywords = [['Sony'],['Nintendo'],['Puma']]
tickers = ['SNE','NTDOY','PUMSY']
stocks = zip(keywords,tickers)

In [11]:
delim = ' '

In [12]:
df_total = pd.read_json('twitter_data.json')

In [13]:
with open('bearer.json') as f:
    bearer = json.load(f)
with open('alpha_vantage_api.json') as f:
    alpha_vantage_api = json.load(f)
key = alpha_vantage_api['key']
token = bearer['access_token']
twitter_url = 'https://api.twitter.com/1.1/tweets/search/30day/dev.json'
header = {'authorization':'Bearer ' + token}

In [14]:
for keyword,ticker in stocks:
    next_token = None
    previous_token = 0
    count = 1
    df_stocks,toDate = stock_Data(ticker,key)
    if df_stocks is not None:
        while True:
            time.sleep(1)
            previous_token = next_token
            df_twitter,next_token= twitter_Data(keyword,next_token,toDate)
            if df_twitter is not None and len(df_twitter) > 0:
                df = df_twitter.merge(df_stocks,on='date')
                if df_total is not None:
                    df_total = pd.concat([df_total,df],axis=0)
                else:
                    df_total = df
                if previous_token == next_token or count == 25:
                    break
                count+= 1
            else:
                break

In [15]:
df_total

,background_image,close,contributors_enabled,date,favorite_count,favorited,geo_enabled,high,low,open,...,urls,user.favourites_count,user.followers_count,user.friends_count,user.listed_count,user.screen_name,user.statuses_count,user_mentions,verified,volume
0,1,195.09,0,1970-01-01 00:00:00.000000001,3,0,0,196.33,189.81,190.02,...,0,35642,64020,11044,1421,Losfinancieros,127149,0,0,51034237
1,0,195.09,0,1970-01-01 00:00:00.000000001,0,0,0,196.33,189.81,190.02,...,0,7466,4264,3648,425,CarlCarano,95931,2,0,51034237
10,1,25.54,0,1970-01-01 00:25:55.459200,0,0,1,25.95,25.415,25.75,...,0,584,508,608,49,ATucci826,28872,2,0,6078311
100,1,25.57,0,1970-01-01 00:25:55.372800,0,0,0,25.67,25.44,25.57,...,0,2435,856,644,3,ShanellMidwest_,95610,0,0,4428696
1000,0,25.57,0,1970-01-01 00:25:55.372800,0,0,0,25.67,25.44,25.57,...,0,2828,1286,2203,1,AndrewL48276395,2106,0,0,4428696
10000,1,191.61,0,1970-01-01 00:25:55.459200,0,0,1,192.35,191.37,192,...,0,4582,598,325,2,f78___,20331,1,0,1875019
100000,1,121.14,0,1970-01-01 00:00:00,0,0,0,121.58,120.57,120.94,...,0,312,48,267,1,S_hiro_designs,224,1,0,10263139
100001,1,1819.26,0,1970-01-01 00:00:00.000000001,0,0,0,1823.75,1787.28,1796.26,...,0,2892,1188,3954,5,9kumichin,45793,2,0,5767797
100002,1,1819.26,0,1970-01-01 00:00:00.000000001,0,0,0,1823.75,1787.28,1796.26,...,0,51720,111,883,6,gmyoheix,22222,1,0,5767797
100003,1,121.14,0,1970-01-01 00:00:00,0,0,0,121.58,120.57,120.94,...,0,2795,38,428,1,n_gdaniels,855,2,0,10263139


In [16]:
idx = pd.Index(range(len(df_total))).unique()

In [17]:
df = pd.DataFrame(df_total.values,columns=list(df_total.columns),index=idx)

In [18]:
df.to_json('twitter_stock_data.json')